In [4]:
from probabilidades import *
import csv
import pandas as pd

PDSexo = ProbDist(
    Sexo_M=4,
    Sexo_F=1,
)

PDIdades = ProbDist(
    {'Idade_A':4,
    'Idade_B':1,
    'Idade_C':0,
    'Idade_D':0}
)



pd_idade_sexo = joint(PDSexo, PDIdades, ' ')

with open('dados/trab_academicos.csv', 'r', encoding='utf8', newline='') as data:
    dados_brutos = pd.read_csv(data, delimiter=',')

In [5]:
#Nomalização de dados

dados = dados_brutos.rename(columns={'Quais dos seguintes ambientes você utiliza na edição de seus trabalhos?': 'ambientes',
                                    'Quais dos seguintes problemas você possui no momento da edição de trabalhos acadêmicos?': 'dificuldades',
                                    'Quais destas opções você gostaria que uma ferramenta de edição fornecesse como ajuda na edição de um trabalho acadêmico?': 'ajuda',
                                     'Com que frequência você realiza trabalhos acadêmicos?': 'frequencia',
                                     'Qual sua formação acadêmica?': 'nivel_ensino'
                                    })

for col in dados.columns:
    dados[col] = dados[col].str.lower()


In [24]:
#Funções de frequencia para montar as distribuições
def frequencia_dado(coluna):
    """Busca as frequencias para todos os valores possíveis da coluna
    param: Coluna a realizar a contagem da frequência
    return: Dicionário com a estrutura Valor:Freq
    """
    freq = {}
    for nivel in dados[coluna].unique():
        freq[nivel] = len(dados.loc[dados[coluna] == nivel])
    return freq

def frequencia_contida(coluna, data_freq):
    """Para perguntas com multiplas escolhas possíveis
    param: Coluna do dados em que deve ocorrer a busca
    param: Lista com os valores a serem buscadas as frêquencias
    """
    freq = {}
    for data in data_freq:
        freq[data] = len(dados.loc[dados[coluna].str.contains(data, case=False)])
    return freq


   


## Pergunta
2. Dado que uma pessoa indique que sua formação acadêmica é Ensino Superior Incompleto qual a probabilidade de ela ter problemas com Criar referências, Fazer citações ou Numeração de páginas?

In [25]:
freq_nivel = frequencia_dado('nivel_ensino')
dist_ensino = ProbDist(freq_nivel)

possivei_problemas = [
    'criação de sumários', 'criação de lista de figuras', 
    'numeração de páginas', 'espaçamento do texto', 'alinhamento do texto', 
    'criar referências', 'fazer citações', 'criar tabelas', 
    'não compreender a ferramenta', 'outros'
]

freq_problemas = frequencia_contida('dificuldades', possivei_problemas)
dist_problemas = ProbDist(freq_problemas)

PDnivel_problema = joint(dist_ensino, dist_problemas, ' ')

#Predicados
def superior_incompleto(r):
    return 'superior incompleto' in r

def  dificuldades(r):
    return 'criar referências' in r or 'fazer citações' in r or 'numeração de páginas' in r


In [26]:
#for i, z in PDnivel_problema.items():
#    Impressão para visualizar o resultado da probabilidade condicional final entre problema e nível de ensino
#    print('{} ---- {}'.format(i, z))

probabilidade_resposta2 = P(superior_incompleto, tal_que(dificuldades , PDnivel_problema))
print("A resposta da pergunta 2 é %.1f%%'" % (probabilidade_resposta2 * 100))

A resposta da pergunta 2 é 73.2%'


## Pergunta
3. Dado que uma pessoa indique que utiliza o ambiente Word qual a probabilidade de que o problema na edição de trabalhos acadêmicos seja Criação de sumários?

In [27]:
possiveis_ferramentas = [
    'google docs', 'word', 'libre office', 'fastformat', 'quip', 'zoho', 'only office', 'outros'
]

freq_ambientes = frequencia_contida('ambientes', possiveis_ferramentas)

dist_ambientes = ProbDist(freq_ambientes)

PDambientes_problemas = joint(dist_ambientes, dist_problemas, ' ')

def word(r):
    return 'word' in r

def sumario(r):
    return 'criação de sumários' in r

In [35]:
probabilidade_resposta3 = P(word, tal_que(sumario, PDambientes_problemas))
print("A resposta da pergunta 3 é %.1f%%'" % (probabilidade_resposta3 * 100))
print(probabilidade_resposta3)


A resposta da pergunta 3 é 47.3%'
0.4732824427480916


## Pergunta
4. Dado que uma pessoa indique que utiliza o ambiente Word e seu problema de edição de trabalhos acadêmicos seja Criação de sumários qual a probabilidade de que ela gostaria que uma ferramenta de edição fornecesse como ajuda a Criação automática de sumários?

In [34]:
possiveis_ajudas = [
    'Modelo de documentos', 'Gerador de Referências', 'Tutorial interativo da Ferramenta',
    'Sugestões de erros', 'Dicas de formatação', 'Instruções para citações',
    'Compartilhar documento com o orientador', 'Gerador de Gráficos', 'Criação automática de sumários',
    'Verificação de plágio', 'Gerador de Tabela de Siglas', 'Gerador de Tabela de Imagens', 'Outros'
]

dist_ajuda = frequencia_contida('ajuda', possiveis_ajudas)
PDAjuda = ProbDist(dist_ajuda)
PDajuda_problema = joint(PDambientes_problemas, PDAjuda, ' ')

def criacao_sumario(r):
    return 'Criação automática de sumários' in r

probabilidade_resposta4 = P(word, tal_que(criacao_sumario, PDajuda_problema))
print(probabilidade_resposta4)

0.4732824427480916
